# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import the city data from the weather section
csv_path = 'city_data.csv'
city_data = pd.read_csv(csv_path)
city_data.head()

,City,Lat,Lon,Max Temperature (f),Humidity,Wind Speed,Clouds,Country,Date
0,hobart,-42.88,147.33,60.80,48,8.05,90,AU,1580077042
1,rikitea,-23.12,-134.97,78.87,79,13.20,0,PF,1580077028
2,hienghene,-20.68,164.93,79.16,80,18.16,7,NC,1580077185
3,pisco,-13.70,-76.22,73.99,30,25.28,48,PE,1580076791
4,mar del plata,-38.00,-57.56,75.20,53,16.11,68,AR,1580076911


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#set up my api key
gmaps.configure(api_key=g_key)
#configure the first map
fig = gmaps.figure()


In [4]:
#make a locations parameter based on the latitude and longitude of the data from the different cities and save as float
locations = city_data[["Lat","Lon"]].astype(float)

#convert the humidity data to dtype float
humidity_map = city_data['Humidity'].astype(float)

#build a heat layer in gmaps to superimpose humidity data over the map using the locations for the city locations and humidity data as the weight.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_map,
                                 dissipating=False, max_intensity=100,point_radius=1)
#add the heat layer to the google map
fig.add_layer(heat_layer)
#print the map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
#build a new data  frame with the specified weather conditions as filters
hotel_df = city_data
#limit to only cities with no clouds
hotel_df = hotel_df[hotel_df['Clouds']<=0]
#limit to cities with a temperature bewteen 70 and 80 degrees f
hotel_df = hotel_df[hotel_df['Max Temperature (f)']<=80]
hotel_df = hotel_df[hotel_df['Max Temperature (f)']>=70]
#limit the data to only cities with wind speed of less than 10mph
hotel_df  = hotel_df[hotel_df['Wind Speed'] <=10]
#drop null values
hotel_df.dropna()

hotel_df.head()
#len(hotel_df["City"])

,City,Lat,Lon,Max Temperature (f),Humidity,Wind Speed,Clouds,Country,Date
7,castro,-24.79,-50.01,79.25,48,0.81,0,BR,1580077037
147,taoudenni,22.68,-3.98,75.69,22,9.98,0,ML,1580076773
228,manyana,-23.40,21.72,73.49,30,7.94,0,BW,1580077221
242,yulara,-25.24,130.99,71.60,24,6.93,0,AU,1580077088
364,srandakan,-7.94,110.25,74.50,91,2.64,0,ID,1580077244


In [42]:
# ### Hotel Map
# * Store into variable named `hotel_df`.
# * Add a "Hotel Name" column to the DataFrame.
# * Set parameters to search for hotels with 5000 meters.
# * Hit the Google Places API for each city's coordinates.
# * Store the first Hotel result into the DataFrame.
# * Plot markers on top of the heatmap.

In [49]:
#make lists for the query data to append to
hotel_names =[]
hotel_lat =[]
hotel_lng =[]
#set the parameters: 5000 meter radius from the city lat/long, searching for hotels in keyword and type, add api key
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
#create a for loop to use iterrows() to loop through the cities in the dataframe and make an api request for each lat/long
for index, row in hotel_df.iterrows():
    #set the lat and lng to the row being interrated
    lat = row["Lat"]
    lng = row["Lon"]
    #update the location parameter to be the current city's lat/lng
    params["location"] = f"{lat},{lng}"
    #update the url to include the parameters
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    #get a json response
    hotel_data = response.json()
    hotel_data=hotel_data["results"]
    #try to append the data from the response
    try:
        hotel_names.append(hotel_data[0]["name"])
        hotel_lat.append(hotel_data[0]['geometry']["location"]['lat'])
        hotel_lng.append(hotel_data[0]['geometry']["location"]['lng'])
        #if no data is returned then append N/A to the list
    except:
        hotel_names.append("N/A")
        hotel_lat.append("N/A")
        hotel_lng.append("N/A")

In [50]:
hotel_names

['Central Palace Hotel - Castro (PR)',
 'N/A',
 'N/A',
 'Desert Gardens Hotel - Ayers Rock Resort',
 'Puri Brata',
 'Mandela hôtel',
 'Desert Prince Hotel',
 'N/A',
 'Hotel Mindongo Ouesso',
 'N/A',
 'Central Hotel',
 'Hotel ibis Styles Iu Namibe']

In [57]:
#add the lists of new data to the existing data frame of cities that match the weather specifications
hotel_df['Hotel Name'] = hotel_names
hotel_df['Hotel Lat'] = hotel_lat
hotel_df['Hotel Lng'] = hotel_lng
hotel_df.head()

,City,Lat,Lon,Max Temperature (f),Humidity,Wind Speed,Clouds,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
7,castro,-24.79,-50.01,79.25,48,0.81,0,BR,1580077037,Central Palace Hotel - Castro (PR),-24.7913,-50.0078
147,taoudenni,22.68,-3.98,75.69,22,9.98,0,ML,1580076773,N/A,N/A,N/A
228,manyana,-23.40,21.72,73.49,30,7.94,0,BW,1580077221,N/A,N/A,N/A
242,yulara,-25.24,130.99,71.60,24,6.93,0,AU,1580077088,Desert Gardens Hotel - Ayers Rock Resort,-25.2437,130.985
364,srandakan,-7.94,110.25,74.50,91,2.64,0,ID,1580077244,Puri Brata,-7.97794,110.269


In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [59]:
# Add marker layer ontop of heat map
#add markers for all the hotesl.  
markers_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers_layer)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))